In [2]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-huggingface sentence-transformers langchain-community
%pip install groq python-dotenv
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_huggingface import HuggingFaceEmbeddings

# Using a FREE, open-source model from Hugging Face
# 'all-MiniLM-L6-v2' is small, fast, and very good for English.
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.3 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from groq import Groq
import os
from getpass import getpass

# Securely accept API key (hidden while typing)
GROQ_API_KEY = getpass("Enter your Groq API Key: ")

client = Groq(api_key=GROQ_API_KEY)

Enter your Groq API Key: ··········


In [4]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a highly skilled Technical Support Engineer. "
            "You specialize in debugging code, analyzing error messages, and providing "
            "precise, step-by-step technical solutions. Be rigorous, code-focused, and exact. "
            "When relevant, provide corrected code snippets."
        ),
        "temperature": 0.7,
    },
    "billing": {
        "system_prompt": (
            "You are an empathetic Billing Support Specialist. "
            "You handle refund requests, subscription issues, and payment disputes with care. "
            "Be understanding, policy-driven, and clear about next steps. "
            "Always acknowledge the customer's frustration before explaining the resolution process."
        ),
        "temperature": 0.7,
    },
    "general": {
        "system_prompt": (
            "You are a friendly and helpful General Support Assistant. "
            "You handle casual inquiries, FAQs, and any questions that don't fall under "
            "technical or billing categories. Be warm, concise, and helpful."
        ),
        "temperature": 0.7,
    },
}

MODEL_NAME = "llama-3.3-70b-versatile"

In [5]:
def route_prompt(user_input: str) -> str:
    """
    Classifies user input into one of: 'technical', 'billing', or 'general'.
    Uses temperature=0 for consistent, deterministic classification.
    Returns ONLY the category name as a lowercase string.
    """
    routing_system_prompt = (
        "You are an intent classification engine. "
        "Classify the user's message into exactly one of these categories: "
        "technical, billing, general. "
        "Rules:\n"
        "- 'technical': bugs, errors, code issues, crashes, API problems\n"
        "- 'billing': charges, refunds, subscriptions, payments, invoices\n"
        "- 'general': everything else\n"
        "Return ONLY the single category word in lowercase. No punctuation, no explanation."
    )

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": routing_system_prompt},
            {"role": "user", "content": user_input},
        ],
        temperature=0,  # Deterministic for routing
        max_tokens=10,
    )

    category = response.choices[0].message.content.strip().lower()

    # Fallback to 'general' if unexpected output
    if category not in MODEL_CONFIG:
        category = "general"

    return category

In [6]:
def process_request(user_input: str) -> str:
    """
    Orchestrates the full MoE pipeline:
    1. Routes the input to the correct expert category.
    2. Loads the corresponding system prompt.
    3. Calls the LLM with that expert configuration.
    4. Returns the expert's response.
    """
    # Step 1: Route to the right expert
    category = route_prompt(user_input)
    print(f"[Router] Classified as: '{category}'")

    # Step 2: Select expert config
    expert_config = MODEL_CONFIG[category]

    # Step 3: Call the LLM with the expert's system prompt
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": expert_config["system_prompt"]},
            {"role": "user", "content": user_input},
        ],
        temperature=expert_config["temperature"],
        max_tokens=512,
    )

    # Step 4: Return the expert's answer
    return response.choices[0].message.content.strip()

In [7]:
query_1 = "My python script is throwing an IndexError on line 5."
print("=" * 60)
print(f"User Query: {query_1}")
print("=" * 60)
answer_1 = process_request(query_1)
print(f"\n[Expert Response]\n{answer_1}\n")

User Query: My python script is throwing an IndexError on line 5.
[Router] Classified as: 'technical'

[Expert Response]
To help you troubleshoot the IndexError, I'll need a bit more information. 

**Please provide the following:**

1. The entire Python script (or at least the relevant section of code around line 5)
2. The full error message, including the line number and the specific error description

With this information, I can help you identify the cause of the IndexError and provide a step-by-step solution to resolve it.

**Example of what I'm looking for:**

```python
# your python script here
```

**And the error message:**

```
IndexError: list index out of range
```

or something similar.

Once I have this information, I can provide a precise and code-focused solution to help you resolve the issue.



In [8]:
query_2 = "I was charged twice for my subscription this month."
print("=" * 60)
print(f"User Query: {query_2}")
print("=" * 60)
answer_2 = process_request(query_2)
print(f"\n[Expert Response]\n{answer_2}\n")

User Query: I was charged twice for my subscription this month.
[Router] Classified as: 'billing'

[Expert Response]
I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you, especially when you're expecting to only be charged once. I'm here to help you resolve this issue as quickly as possible.

To start, I'd like to look into this further and see what happened. Can you please confirm your subscription details with me, such as the date of the duplicate charge and the amount that was taken out? Additionally, have you noticed any other discrepancies with your account or subscription?

Once I have this information, I'll be happy to assist you with a refund for the duplicate charge. Please know that our policy is to promptly refund any incorrect or duplicate charges, and I'll work with you to ensure that this is taken care of. I appreciate your patience and understanding as we work through this issue together.



In [9]:
query_3 = "What are your business hours?"
print("=" * 60)
print(f"User Query: {query_3}")
print("=" * 60)
answer_3 = process_request(query_3)
print(f"\n[Expert Response]\n{answer_3}\n")

User Query: What are your business hours?
[Router] Classified as: 'general'

[Expert Response]
Our business hours are Monday to Friday, 9:00 AM to 5:00 PM (EST). We're closed on weekends and major holidays. If you have any questions or need assistance outside of these hours, you can always leave a message and we'll get back to you as soon as possible during our next business day. How can I assist you today?



BONUS

In [10]:
def get_mock_bitcoin_price() -> str:
    """Mock function simulating a live data fetch for Bitcoin price."""
    # In a real scenario, this would call an API like CoinGecko
    mock_price = 67_432.15
    return f"The current price of Bitcoin (BTC) is ${mock_price:,.2f} USD (mock data)."


def route_prompt_with_tool(user_input: str) -> str:
    """
    Extended router that also detects 'tool' intent for live data queries.
    Returns one of: 'technical', 'billing', 'general', 'tool'.
    """
    routing_system_prompt = (
        "You are an intent classification engine. "
        "Classify the user's message into exactly one of these categories: "
        "technical, billing, general, tool. "
        "Rules:\n"
        "- 'technical': bugs, errors, code issues, crashes\n"
        "- 'billing': charges, refunds, subscriptions, payments\n"
        "- 'tool': requests for live/real-time data like cryptocurrency prices, stock prices, weather\n"
        "- 'general': everything else\n"
        "Return ONLY the single category word in lowercase. No punctuation, no explanation."
    )

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": routing_system_prompt},
            {"role": "user", "content": user_input},
        ],
        temperature=0,
        max_tokens=10,
    )

    category = response.choices[0].message.content.strip().lower()
    if category not in list(MODEL_CONFIG.keys()) + ["tool"]:
        category = "general"
    return category


def process_request_with_tool(user_input: str) -> str:
    """
    Extended orchestrator that handles the 'tool' expert by calling
    a mock data-fetching function instead of an LLM.
    """
    category = route_prompt_with_tool(user_input)
    print(f"[Router] Classified as: '{category}'")

    # Handle tool use separately — call the function, not the LLM
    if category == "tool":
        if "bitcoin" in user_input.lower():
            return get_mock_bitcoin_price()
        return "Tool use requested, but no matching tool was found."

    # Otherwise, use the normal expert pipeline
    expert_config = MODEL_CONFIG[category]
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": expert_config["system_prompt"]},
            {"role": "user", "content": user_input},
        ],
        temperature=expert_config["temperature"],
        max_tokens=512,
    )
    return response.choices[0].message.content.strip()


# Test the Tool Use Expert
query_4 = "What is the current price of Bitcoin?"
print("=" * 60)
print(f"User Query: {query_4}")
print("=" * 60)
answer_4 = process_request_with_tool(query_4)
print(f"\n[Expert Response]\n{answer_4}\n")

User Query: What is the current price of Bitcoin?
[Router] Classified as: 'tool'

[Expert Response]
The current price of Bitcoin (BTC) is $67,432.15 USD (mock data).

